# Pytorch+CUDA Setup and basic playground

In [1]:
# Install Pytorch with CUDA

!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu124


In [2]:
# Check Pytorch and CUDA versions

import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())
print(torch.cuda.get_arch_list())

2.5.1+cu124
12.4
True
['sm_50', 'sm_60', 'sm_61', 'sm_70', 'sm_75', 'sm_80', 'sm_86', 'sm_90']


In [3]:
# Playing with Tensors

# Create a tensor
x = torch.tensor([1, 2, 3])
print(x)

# Create a random tensor
random_tensor = torch.rand(3, 3)
print(random_tensor)

# Move tensor to GPU (if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x = x.to(device)


tensor([1, 2, 3])
tensor([[0.8042, 0.8935, 0.9075],
        [0.7240, 0.8360, 0.2140],
        [0.2623, 0.9995, 0.0457]])


# Simple Neural Network for MNIST

In [4]:
# Creating a Simple Neural Network

import torch.nn as nn

class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = SimpleNN()


In [5]:
# Preparing Data (DataLoader)

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)


In [7]:
# Training the Neural Network

import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 5
for epoch in range(num_epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data.view(data.size(0), -1))
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print(f'Epoch {epoch+1}/{num_epochs}, Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item():.4f}')

Epoch 1/5, Batch 0/938, Loss: 2.3150
Epoch 1/5, Batch 100/938, Loss: 0.5943
Epoch 1/5, Batch 200/938, Loss: 0.5443
Epoch 1/5, Batch 300/938, Loss: 0.3057
Epoch 1/5, Batch 400/938, Loss: 0.4579
Epoch 1/5, Batch 500/938, Loss: 0.2955
Epoch 1/5, Batch 600/938, Loss: 0.3502
Epoch 1/5, Batch 700/938, Loss: 0.1585
Epoch 1/5, Batch 800/938, Loss: 0.3299
Epoch 1/5, Batch 900/938, Loss: 0.2208
Epoch 2/5, Batch 0/938, Loss: 0.1555
Epoch 2/5, Batch 100/938, Loss: 0.2456
Epoch 2/5, Batch 200/938, Loss: 0.3014
Epoch 2/5, Batch 300/938, Loss: 0.2958
Epoch 2/5, Batch 400/938, Loss: 0.1465
Epoch 2/5, Batch 500/938, Loss: 0.2475
Epoch 2/5, Batch 600/938, Loss: 0.1831
Epoch 2/5, Batch 700/938, Loss: 0.1366
Epoch 2/5, Batch 800/938, Loss: 0.2686
Epoch 2/5, Batch 900/938, Loss: 0.0826
Epoch 3/5, Batch 0/938, Loss: 0.1343
Epoch 3/5, Batch 100/938, Loss: 0.1888
Epoch 3/5, Batch 200/938, Loss: 0.0379
Epoch 3/5, Batch 300/938, Loss: 0.0937
Epoch 3/5, Batch 400/938, Loss: 0.0609
Epoch 3/5, Batch 500/938, Loss:

In [8]:
# Evaluating the model on test dataset

model.eval()
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        output = model(data.view(data.size(0), -1))
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

print(f'Accuracy on test set: {100 * correct / total:.2f}%')

Accuracy on test set: 96.74%


In [9]:
# Save the model
torch.save(model.state_dict(), 'model.pth')

In [10]:
# Load the model
loaded_model = SimpleNN()
loaded_model.load_state_dict(torch.load('model.pth'))
loaded_model.eval()

C:\Users\apoor\AppData\Local\Temp\ipykernel_23360\3081489078.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model.load_state_dict(torch.load('model.pth'))


SimpleNN(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
  (relu): ReLU()
)